In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests


In [3]:
URL ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Tilastopaja_Asian_Games_2018.html'  # open a locally saved copy of html
HTMLFileToBeOpened = open(URL, "r") 
contents = HTMLFileToBeOpened.read() 
soup = BeautifulSoup(contents, 'lxml') 


In [204]:
print(soup.body.prettify()) 


<body onload="document.getElementById('home').className='active'; leader(); SetLoginFocus();">
 <script>
  function load(){
	document.getElementById("siteLoader").style.display="block";
}
 </script>
 <div class="wrapper">
  <div class="top">
   <div class="logo">
    <h1>
     <a ref="https://www.tilastopaja.info">
      TILASTOPAJA
     </a>
    </h1>
   </div>
   <div class="container">
    <ul id="nav">
     <li class="haku">
      <div>
       SEARCH FOR AN ATHLETE
       <form action="https://www.tilastopaja.info/db/ats.php" method="GET">
        <input class="text" name="Name" size="20" style="font-weight: normal;"/>
        <input class="button" type="submit" value="Go!"/>
       </form>
      </div>
     </li>
     <li id="home">
      <a href="https://www.tilastopaja.info/">
       Home
      </a>
     </li>
     <li>
      <a href="#s1">
       Free
      </a>
      <span id="s1">
      </span>
      <ul class="subs">
       <li>
        <a href="https://www.tilastopaja.info/

In [312]:
# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A> (name) or <B> (heats and stage)

lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
dict_result2=[]  # length 2 dict
row=[]

temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT'])

h1s = soup.find_all('h1')

for h1 in h1s:
    
    print(h1.text)


for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
        print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
        
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                
                if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           
                    
                    names.append(n.text) # add 'clean' name to list
                              
            for m in value.find_all('b'):  # extract heats and stage
                                
                print('MMMMM', m.text)
                 
          #  for count_b, value_b in enumerate(value.find_all('b')):  # extract the name and put into a list
                                
          #      print('MMMMM', value_b.text)

                                       
            if len(lists)==2:
                
                heat=''
                wind=''
                dict_result=''
                dict_result2=''
                stage=''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
            if 'Date' in value.text:
                date=value.text
                          
            if 'Wind' in value.text:
                wind=value.text
     #           print('WINDDDDD', value.text)
                
            print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            if 'Wind' in lists[2]:
                
                wind = lists[2][5:]  # slice out the 'Wind:'
            
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
            if len(lists[2])>1:
                                
                date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
            date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
            
                        
        if len(lists)==11: # dict length 10 contains athlete info
                        
            dict_result=lists
                        
            
                        
        #    dict_result2 = [] # reset attempts
            
                        
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
            
                    
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
        print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        try:
            
            if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:
                print('dict_result2', dict_result2)
                
                if len(dict_result2)>0:
                          
                    
                    row.append(dict_result[0]) # rank
                    for k in range(0, len(names)):  #search and replace name
                        
                        hh=''
                    
                        if names[k] in dict_result[1]:
                            
                            print('XXX', names[k], dict_result[1])
                            
                            hh=names[k]

                    row.append(hh) # name  (dict_result[1])

                 #   row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append('18th Asian Games')
                    row.append('2018')
                    row.append(date)
                    row.append(event)
                    row.append('Gelora Bung Karno Stadium')
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                                        
                else:
                    
                    pass

                    
            
            elif (event and dict_result[3] and dict_result[5]) is not None:
                                    
               #

                row.append(dict_result[0]) # rank
               # row.append(names[1]) # name 
                
                for k in range(0, len(names)):  #search and replace name
                    
                    hh=''
                    
                    if names[k] in dict_result[1]:
                    
                        hh=names[k]

                row.append(hh) # name  (dict_result[1])

               # row.append(dict_result[1]) # name  (dict_result[1])
                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append('18th Asian Games')
                row.append('2018')
                row.append(date)
                row.append(event)
                row.append('Gelora Bung Karno Stadium')
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB                    
                row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT
                
               

            #print(row)
            
        except:
            
            pass
        
        if len(row)>5:
                    
            temp_df.loc[len(temp_df)] = row

      #  print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
            
            
        
        
    wind=''
    heat=''
    date=''
    
    lists=[]
    dict_result = []
    row=[]
                        
            
                
            
            
            


        
      

TILASTOPAJA
RESULTS: 18th Asian Games, Gelora Bung Karno Stadium, Jakarta, INA  - 25 - 30 August 2018 
LATEST MAJOR RESULTS
▼LATEST MAJOR RESULTS
NEW SECTION
TDTD 0 
 Men's events: 


Please choose men's event
Men  100m
Men  200m
Men  400m
Men  800m
Men  1500m
Men  5000m
Men  10,000m
Men  Marathon
Men  3000m Steeplechase
Men  110m Hurdles
Men  400m Hurdles
Men  High Jump
Men  Pole Vault
Men  Long Jump
Men  Triple Jump
Men  Shot Put
Men  Discus Throw
Men  Hammer Throw
Men  Javelin Throw
Men  Decathlon
Men  20km Race Walk
Men  50km Race Walk
Men  4 x 100m
Men  4 x 400m
Men  Mixed Relay
Men Multievents 100m
Men Multievents 400m
Men Multievents 1500m
Men Multievents 110m Hurdles
Men Multievents High Jump
Men Multievents Pole Vault
Men Multievents Long Jump
Men Multievents Shot Put
Men Multievents Discus Throw
Men Multievents Javelin Throw



TDTD 1      
TDTD 2 
 Women's events: 


Please choose women's event
Women  100m
Women  200m
Women  400m
Women  800m
Women  1500m
Women  5000m
Women  

NEW SECTION
TDTD 0 2
TDTD 1 Zhao Changhong Zhao Changhong  ('96)
TDTD 2 
TDTD 3 CHN
TDTD 4 27 Dec 96
TDTD 5 30:07.49 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 9 
TDTD 10  
EVENT 10,000m     GENDER Male STAGE   HEAT   WIND   DATE 26 August DICT_RESULT {0: '2', 1: "Zhao Changhong\xa0Zhao Changhong\xa0\xa0('96)", 2: '', 3: 'CHN', 4: '27 Dec 96', 5: '30:07.49 ', 6: '', 7: '\xa0', 8: '« »', 9: '', 10: ' '} ATTEMPTS []
NEW SECTION
TDTD 0 3
TDTD 1 Kieran TuntivateKieran Tuntivate ('97)
TDTD 2 
TDTD 3 THA
TDTD 4 16 Feb 97
TDTD 5 30:29.04 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 9 
TDTD 10  
EVENT 10,000m     GENDER Male STAGE   HEAT   WIND   DATE 26 August DICT_RESULT {0: '3', 1: "Kieran\xa0TuntivateKieran\xa0Tuntivate\xa0('97)", 2: '', 3: 'THA', 4: '16 Feb 97', 5: '30:29.04 ', 6: '', 7: '\xa0', 8: '« »', 9: '', 10: ' '} ATTEMPTS []
NEW SECTION
TDTD 0 4
TDTD 1 Zhu Renxue Zhu Renxue  ('91)
TDTD 2 
TDTD 3 CHN
TDTD 4 6 Apr 91
TDTD 5 30:57.13 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 9 
TDTD 10  
EVENT 10,000m     GENDE

NEW SECTION
TDTD 0 4
TDTD 1 Dana AbdulrazakDana Abdulrazak ('86)
TDTD 2 D 2y 6m 
TDTD 3 IRQ
TDTD 4 3 Jan 86
TDTD 5 24.31 
TDTD 6 
TDTD 7  
TDTD 8 SB «
TDTD 9 
TDTD 10  
EVENT 200m     GENDER Female STAGE Heats HEAT Heat 4 WIND  -0.1 DATE 28 August DICT_RESULT {0: '4', 1: "Dana\xa0AbdulrazakDana\xa0Abdulrazak\xa0('86)", 2: 'D 2y 6m ', 3: 'IRQ', 4: '3 Jan 86', 5: '24.31 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS []
NEW SECTION
TDTD 0 5
TDTD 1 Alfina TehupeioryAlfina Tehupeiory ('95)
TDTD 2 
TDTD 3 INA
TDTD 4 5 Apr 95
TDTD 5 24.36 
TDTD 6 
TDTD 7  
TDTD 8 PB »
TDTD 9 
TDTD 10  
EVENT 200m     GENDER Female STAGE Heats HEAT Heat 4 WIND  -0.1 DATE 28 August DICT_RESULT {0: '5', 1: "Alfina\xa0TehupeioryAlfina\xa0Tehupeiory\xa0('95)", 2: '', 3: 'INA', 4: '5 Apr 95', 5: '24.36 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: ' '} ATTEMPTS []
NEW SECTION
TDTD 0 6
TDTD 1 Poon Hang Wai Poon Hang Wai  ('98)
TDTD 2 
TDTD 3 HKG
TDTD 4 28 Apr 98
TDTD 5 25.59 
TDTD 6 
TDTD 7  
TDTD 8 « »
TDTD 

NEW SECTION
TDTD 0 10
TDTD 1 Nayana JamesNayana James ('95)
TDTD 2 
TDTD 3 IND
TDTD 4 18 Oct 95
TDTD 5 6.14 
TDTD 6 0.0
TDTD 7  
TDTD 8 « »
TDTD 9 
TDTD 10  
EVENT Long Jump     GENDER Female STAGE  HEAT  WIND  DATE 27 August DICT_RESULT {0: '10', 1: "Nayana\xa0JamesNayana\xa0James\xa0('95)", 2: '', 3: 'IND', 4: '18 Oct 95', 5: '6.14 ', 6: '0.0', 7: '\xa0', 8: '« »', 9: '', 10: ' '} ATTEMPTS 
dict_result2 
NEW SECTION
TDTD 0  
TDTD 1   6.08/-0.2        6.14        X/-0.1      
EVENT Long Jump     GENDER Female STAGE  HEAT  WIND  DATE 27 August DICT_RESULT {0: '10', 1: "Nayana\xa0JamesNayana\xa0James\xa0('95)", 2: '', 3: 'IND', 4: '18 Oct 95', 5: '6.14 ', 6: '0.0', 7: '\xa0', 8: '« »', 9: '', 10: ' '} ATTEMPTS {0: '\xa0', 1: '  6.08/-0.2        6.14        X/-0.1      '}
dict_result2 {0: '\xa0', 1: '  6.08/-0.2        6.14        X/-0.1      '}
XXX Nayana James Nayana JamesNayana James ('95)
NEW SECTION
TDTD 0 11
TDTD 1 Anna BulanovaAnna Bulanova ('94)
TDTD 2 
TDTD 3 KGZ
TDTD 4 12 May 9

NEW SECTION
TDTD 0  
MMMMM Shot Put
TDTD 1 Shot Put    
TDTD 2 28 August
TDTD 3  
MMMMM 
TDTD 4   
EVENT Shot Put     GENDER Female STAGE   HEAT   WIND   DATE 28 August DICT_RESULT [] ATTEMPTS []
dict_result2 []
NEW SECTION
TDTD 0 1
TDTD 1 Shen Muhan Shen Muhan  ('97)
TDTD 2 
TDTD 3 CHN
TDTD 4 5 Jan 97
TDTD 5 12.78 
TDTD 6 
TDTD 7  
TDTD 8 SB «
TDTD 9 
TDTD 10  
EVENT Shot Put     GENDER Female STAGE   HEAT   WIND   DATE 28 August DICT_RESULT {0: '1', 1: "Shen Muhan\xa0Shen Muhan\xa0\xa0('97)", 2: '', 3: 'CHN', 4: '5 Jan 97', 5: '12.78 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS []
dict_result2 []
NEW SECTION
TDTD 0  
TDTD 1   11.93        12.26        12.78    SB  
EVENT Shot Put     GENDER Female STAGE   HEAT   WIND   DATE 28 August DICT_RESULT {0: '1', 1: "Shen Muhan\xa0Shen Muhan\xa0\xa0('97)", 2: '', 3: 'CHN', 4: '5 Jan 97', 5: '12.78 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS {0: '\xa0', 1: '  11.93        12.26        12.78    SB  '}
dict_result2 {0: '

In [305]:
len(names)

1070

In [306]:
names

['Su Bingtian\xa0',
 'Tosin\xa0Ogunode',
 'Ryota\xa0Yamagata',
 'Abdullah Abkar\xa0Mohammed',
 'Yang Chun-Han\xa0',
 'Hassan\xa0Taftian',
 'Lalu Muhammad\xa0Zohri',
 'Kim Kuk-Young\xa0',
 'Abdullah Abkar\xa0Mohammed',
 'Lalu Muhammad\xa0Zohri',
 'Xu Zhouzheng\xa0',
 'Andrew\xa0Fisher',
 'Wang Wei-Hsu\xa0',
 'Bandit\xa0Chuangchai',
 'Ng Ka Fung\xa0',
 'Hassan\xa0Saaid',
 'Su Bingtian\xa0',
 'Tosin\xa0Ogunode',
 'Aska\xa0Cambridge',
 'Jonathan\xa0Nyepa',
 'Oh Kyung-Su\xa0',
 'Yaspy\xa0Bobi',
 'Tsui Chi Ho\xa0',
 'Gohar\xa0Shahbaz',
 'Ryota\xa0Yamagata',
 'Yang Chun-Han\xa0',
 'Hassan\xa0Taftian',
 'Kim Kuk-Young\xa0',
 'Barakat\xa0Al-Harthi',
 'Khairul Hafiz\xa0Jantan',
 'Chayut\xa0Khongprasit',
 'Saeed\xa0Al-Khaldi',
 'Tosin\xa0Ogunode',
 'Ryota\xa0Yamagata',
 'Kim Kuk-Young\xa0',
 'Bandit\xa0Chuangchai',
 'Liaqat\xa0Ali',
 'Sokong\xa0Pen',
 'Vitaliy\xa0Zems',
 'Yang Chun-Han\xa0',
 'Aska\xa0Cambridge',
 'Tsui Chi Ho\xa0',
 'Chayut\xa0Khongprasit',
 'Saeed\xa0Al-Khaldi',
 'Fatik\xa0Qisa

In [307]:
namess=pd.DataFrame(names)

In [308]:
namess.to_csv('names.csv', encoding='utf-8')

In [309]:
temp_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Su Bingtian,CHN,9.92,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,29 Aug 89,« »,,[]
1,2,Tosin Ogunode,QAT,10.00,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,2 Mar 94,PB »,,[]
2,3,Ryota Yamagata,JPN,10.00,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,10 Jun 92,=PB »,,[]
3,4,Abdullah Abkar Mohammed,KSA,10.10,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,1 Jun 97,« »,,[]
4,5,Yang Chun-Han,TPE,10.17,,18th Asian Games,2018,26 August,100m,Gelora Bung Karno Stadium,Male,,,0.8,1 Jan 97,« »,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,6,Meg Hemphill,JPN,43.71,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,23 May 96,« »,,"{0: ' ', 1: ' 43.71 39.89 43.19..."
1142,7,Chu Chia-Ling,TPE,42.74,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,11 Jan 91,SB «,,"{0: ' ', 1: ' 39.39 42.28 SB 42...."
1143,8,Wang Qingling,CHN,39.33,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,14 Jan 93,« »,,"{0: ' ', 1: ' 35.60 38.21 39.33..."
1144,9,Jeong Yeon-Jin,KOR,35.64,,18th Asian Games,2018,29 August,Javelin Throw,Gelora Bung Karno Stadium,Female,,,,25 Mar 92,« »,,"{0: ' ', 1: ' 35.64 33.09 31.26..."


In [310]:
columns =  list(temp_df.columns) 
    
for col in columns:
    temp_df[col] = temp_df[col].astype(str)
    temp_df[col] = temp_df[col].str.replace('\xa0', ' ', regex=True)

In [311]:
temp_df.to_csv("2018_AG.csv", encoding='utf-8')

# 17th Asian Games Incheon

In [128]:
URL2 ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Tilastopaja - 17th Asian Games - Incheon.html'  # open a locally saved copy of html
HTMLFileToBeOpened = open(URL2, "r") 
contents = HTMLFileToBeOpened.read() 
soup2 = BeautifulSoup(contents, 'lxml') 


In [131]:
# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A>

lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
dict_result2=[]  # length 2 dict
row=[]

df_incheon = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT'])


h1s = soup2.find_all('h1')

for h1 in h1s:
    
    print(h1.text)


for i in soup2.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
    #    print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                                
                names.append(n.text)
                
                                   
                     
            if len(lists)==2:
                
                heat=''
                wind=''
                dict_result=''
                dict_result2=''
                stage=''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
            if 'Date' in value.text:
                date=value.text
                          
            if 'Wind' in value.text:
                wind=value.text
     #           print('WINDDDDD', value.text)
                
    #        print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
          
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
            if len(lists[2])>1:
                                
                date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
            date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
            
                        
        if len(lists)==11: # dict length 10 contains athlete info
                        
            dict_result=lists
            
            dict_results2 = [] # reset attempts
              
                
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
     #   print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        print('NAMES', names)
        try:
            
            if (dict_result[0] and dict_result[1] and event) is not None:
                        
                row.append(dict_result[0]) # rank
                row.append(dict_result[1]) # name  (dict_result[1])
               # row.append(names[1]) # name  (dict_result[1])

                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append('17th Asian Games')
                row.append('2014')
                row.append(date)
                row.append(event)
                row.append('Incheon Asiad Main Stadium')
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB                    
                row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT


            #print(row)
            
        except:
            
            pass
        
        if len(row)!=0:
        
            df_incheon.loc[len(df_incheon)] = row

      #  print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
        
    wind=''
    heat=''
    date=''
    
    lists=[]
    dict_result = []
    row=[]
                        
                
                
            
            

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [132]:
df_incheon

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Femi OgunodeFemi Ogunode ('91),QAT,9.93,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,15 May 91,PB »,0.184,[]
1,2,Su Bingtian Su Bingtian ('89),CHN,10.10,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,29 Aug 89,SB «,0.145,[]
2,3,Kei TakaseKei Takase ('88),JPN,10.15,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,25 Nov 88,« »,0.152,[]
3,4,Zhang Peimeng Zhang Peimeng ('87),CHN,10.18,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,13 Mar 87,« »,0.128,[]
4,5,Reza GhasemiReza Ghasemi ('87),IRI,10.25,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,24 Jul 87,SB «,0.161,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,7,Sushmita Singha RoySushmita Singha Roy ('84),IND,36.91,,17th Asian Games,2014,29 September,Javelin Throw,Incheon Asiad Main Stadium,Female,,,,26 Mar 84,« »,,[]
1318,8,Kotchakorn KhamrueangsriKotchakorn Khamrueangs...,THA,36.30,,17th Asian Games,2014,29 September,Javelin Throw,Incheon Asiad Main Stadium,Female,,,,13 Mar 95,« »,,[]
1319,9,Huang Yu-Ting Huang Yu-Ting ('88),TPE,34.43,,17th Asian Games,2014,29 September,Javelin Throw,Incheon Asiad Main Stadium,Female,,,,17 Nov 88,SB «,,[]
1320,10,Jeong Yeon-Jin Jeong Yeon-Jin ('92),KOR,30.79,,17th Asian Games,2014,29 September,Javelin Throw,Incheon Asiad Main Stadium,Female,,,,25 Mar 92,« »,,[]
